In [ ]:
import mne
import numpy as np
import os


data_root = "YOUR_DATA_ROOT"
output_root = "YOUR_OUTPUT_ROOT"

for class_name in ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']:
    for split in ['train', 'test']:
        vhdr_file_path = f'{data_root}/{class_name}_{split}.vhdr'
        print(vhdr_file_path)
        # Load the data
        raw = mne.io.read_raw_brainvision(vhdr_file_path, preload=True)
        print(raw.info)

        # Apply a band-pass filter
        raw.filter(l_freq=1.0, h_freq=40.0)  # Filters the data to keep frequencies between 1 and 40 Hz

        # Extract events
        events, event_id = mne.events_from_annotations(raw)

        # Define epochs parameters: tmin and tmax are the start and end times relative to each event
        tmin, tmax = -0.05, 0.05  # seconds

        # Create epochs
        epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, preload=True)

        data = epochs.get_data()
        for i, epoch_data in enumerate(data):
            filename = f'{output_root}/{split}/{class_name}/epoch_{i+1}_eventid_{epochs.events[i, 2]}.npy'
            if not os.path.exists(f'{output_root}/{split}/{class_name}'):
                os.makedirs(f'{output_root}/{split}/{class_name}')
            np.save(filename, epoch_data)

            print(f'Saved {filename}')

